

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k, device=x.device)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [2]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.gru = nn.GRU(input_size=N+1, hidden_size=hidden_size, batch_first=True)
    self.decoder = nn.Linear(hidden_size, N+1)

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    out, _ = self.gru(x)
    logits = self.decoder(out)
    return logits

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z.
    # You need to map those characters to one-hot encodings,
    # then get the result from your network, and then convert the output
    # back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    device = next(self.parameters()).device

    # Map string -> integer tensor
    seq = torch.tensor(
        [ord(c) - ord('a') + 1 for c in s], dtype=torch.long, device=device
    ).unsqueeze(0)  # shape: (1, seq_len)
    # One-hot encode
    onehot = idx_to_onehot(seq)  # (1, seq_len, N+1)
    onehot = onehot.to(device).float()
    # Forward pass
    logits = self.forward(onehot)  # (1, seq_len, N+1)
    # Argmax over vocabulary
    preds = torch.argmax(logits, dim=-1).squeeze(0).tolist()  # list of ints
    # Convert back: 0 -> ' ', 1–26 -> 'a'..'z'
    inv_map = [' '] + list(string.ascii_lowercase)
    out_str = ''.join(inv_map[i] for i in preds)

    return out_str

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [3]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [4]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

start_time = time.time()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Epochs and Batch Size
epochs = 5
batch_size = 128

# Initialize model
model = GRUMemory(hidden_size=64).to(device)

# Loss, optimizer and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# Initialize training dataloader
train_loader = DataLoader(ds, batch_size=batch_size)

epoch_progress_bar = tqdm(range(epochs), desc="Training", position=0)
# Training
for epoch in epoch_progress_bar:
    model.train()
    running_loss = 0.0
    total_tokens = 0
    correct_tokens = 0

    for seq_batch, target_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", position=1, leave=False):
        seq_batch = seq_batch.to(device)           # (B, seq_len)
        target_batch = target_batch.to(device)     # (B, seq_len)

        # One-hot encode input
        x = idx_to_onehot(seq_batch).float()       # (B, seq_len, N+1)

        # Forward pass
        logits = model(x)                          # (B, seq_len, N+1)
        loss = criterion(logits.view(-1, N+1), target_batch.view(-1))

        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # Track running loss and accuracy
        running_loss += loss.item() * seq_batch.size(0)  # sum over batch
        preds = logits.argmax(dim=-1)
        correct_tokens += (preds == target_batch).sum().item()
        total_tokens += seq_batch.numel()

    # Average metrics for the epoch
    avg_loss = running_loss / len(ds)
    avg_acc = correct_tokens / total_tokens

    # Update outer tqdm description
    epoch_progress_bar.set_description(f"Epoch {epoch+1}/{epochs} | Avg Loss: {avg_loss:.4f} | Acc: {avg_acc:.4f}")

    # Step scheduler per epoch
    scheduler.step()

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, f'execution took {duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Training:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/1563 [00:00<?, ?it/s]

tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [5]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [6]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    self.hidden_size = hidden_size
    self.max_delay = max_delay
    self.gru = nn.GRU(input_size=N+1+max_delay+1, hidden_size=hidden_size, batch_first=True)
    self.decoder = nn.Linear(hidden_size, N+1)

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    B, seq_len, _ = x.shape
    # Expand delays to all timesteps: (B, seq_len, 1)
    delay_onehot = torch.zeros(B, self.max_delay + 1, device=x.device)
    delay_onehot.scatter_(1, delays.unsqueeze(1).long(), 1.0)

    # Expand to all timesteps: (B, max_delay+1) -> (B, seq_len, max_delay+1)
    delay_onehot_expanded = delay_onehot.unsqueeze(1).expand(-1, seq_len, -1)

    # Concatenate with input
    gru_input = torch.cat([x, delay_onehot_expanded], dim=-1)  # (B, seq_len, N+1+max_delay+1)

    h0 = torch.zeros(1, B, self.hidden_size, device=x.device)
    gru_out, _ = self.gru(gru_input, h0)
    logits = self.decoder(gru_out)  # (B, seq_len, N+1)
    return logits

  @torch.no_grad()
  def test_run(self, s, delay):
      self.eval()
      device = next(self.parameters()).device

      # Map string to integers (a->1, b->2, ..., z->26)
      int_seq = torch.tensor(
          [ord(c) - ord('a') + 1 for c in s],
          dtype=torch.int64,
          device=device
      ).unsqueeze(0)  # (1, seq_len)

      # One-hot encode
      x = idx_to_onehot(int_seq).float().to(device)  # (1, seq_len, N+1)

      # Delay tensor
      delay_tensor = torch.tensor([delay], device=device)

      # Forward pass
      logits = self.forward(x, delay_tensor)  # (1, seq_len, N+1)

      # Get predictions
      pred_ids = logits.argmax(dim=-1).squeeze(0).cpu().tolist()

      # Convert to characters: 0 -> ' ', 1-26 -> 'a'-'z'
      result = []
      for idx in pred_ids:
          if idx == 0:
              result.append(' ')
          else:
              result.append(chr(idx - 1 + ord('a')))

      return ''.join(result)

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [7]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [8]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Epochs and Batch Size
epochs = 10
batch_size = 128

# Initialize model
model = VariableDelayGRUMemory(hidden_size=256, max_delay=MAX_DELAY).to(device)

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# Dataset and dataloader
ds_var = VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size=200_000)
train_loader = DataLoader(ds_var, batch_size=batch_size)

epoch_progress_bar = tqdm(range(epochs), desc="Training", position=0)
for epoch in epoch_progress_bar:
    model.train()
    running_loss = 0.0
    total_tokens = 0
    correct_tokens = 0

    # Inner progress bar per batch
    for seq_batch, delays_batch, target_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", position=1, leave=False):
        seq_batch = seq_batch.to(device)
        target_batch = target_batch.to(device)
        delays_batch = delays_batch.to(device)

        # One-hot encode input
        x = idx_to_onehot(seq_batch).float()

        # Forward pass
        logits = model(x, delays_batch)
        loss = criterion(logits.view(-1, N+1), target_batch.view(-1))

        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Track metrics
        running_loss += loss.item() * seq_batch.size(0)
        preds = logits.argmax(dim=-1)
        correct_tokens += (preds == target_batch).sum().item()
        total_tokens += seq_batch.numel()

    avg_loss = running_loss / len(ds_var)
    avg_acc = correct_tokens / total_tokens

    # Update outer tqdm
    epoch_progress_bar.set_description(f"Epoch {epoch+1}/{epochs} | Avg Loss: {avg_loss:.4f} | Acc: {avg_acc:.4f}")

    # Step scheduler
    scheduler.step()


end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 2/10:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 3/10:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 4/10:   0%|          | 0/1563 [00:00<?, ?it/s]

...

Epoch 7/10:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 8/10:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 9/10:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch 10/10:   0%|          | 0/1563 [00:00<?, ?it/s]

tests passed
